# Математическое моделирование в ORtools: задача планирования расписаний

Приветствую! Я Ложкинс Алексей, консультант и разработчик математических моделей и оптимизационных решений для бизнеса. Это вторая обучающая статья, часть личного образовательного проекта "Make optimization simple". Цель проекта - продемонстрировать доступность технологий и показать на примерах, что моделировать можно без глубокого математического фундамента.

В статье рассмотрим задачу [планирования расписаний](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B8%D1%8F_%D1%80%D0%B0%D1%81%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D0%B9), в частности, планирование смен сотрудников сети стоматологических клиник. Я покажу, как реализовать программный прототип математической модели посредством Python и библиотеки [OR-Tools](https://developers.google.com/optimization?hl=ru), а также продемонстрирую, как получить решение задачи всего в одной строке кода.

Материал статьи предназначен для

- базового погружения в математическое моделирование;
- демонстрации доступности инструментов математического моделирования; 
- погружения в простоту и изящество процесса решения математических задач готовыми решателями.

[<= Предыдущая статья](https://habr.com/ru/articles/731006/)


## Описание задачи

Одной из распространенных задач планирования расписаний является распределение сотрудников по сменам. В качестве примера рассмотрим задачу планирования графика работы сотрудников в сети стоматологических клиник. 

В задаче сделаем акцент на ведущих стоматологах сети как наиболее ценных и редких кадрах.  Такого рода специалисты работают в нескольких клиниках сети одновременно. Распределение ведущих специалистов позволяет клиникам предлагать широкий спектр специалистов, реализующих услуги "повышенного" качества. При этом в штате можно иметь небольшое количество высококвалифицированных специалистов. 

<!-- Качество - это дополнительный атрибут услуги, который также продается. Одной из составляющих качества является квалификация сотрудника, который выполняет работу. Например, ведущий специалист, главный специалист, senior разработчик. 
Компании по предоставлению услуг используют качество для расширения ассортимента услуг. Стоматологические клиники не исключение. Однако, предложение экспертов на рынке труда ограничено, поэтому использовать такой ресурс приходится эффективнее. -->

Перейдем непосредственно к моделируемой проблеме. Рассмотрим задачу распределения ограниченного количества ведущих стоматологов в сети клиник. С целью предоставления клиентам выбора из нескольких ведущих экспертов-стоматологов к каждой из клиник необходимо привязать несколько ведущих сотрудников. Один эксперт может работать в нескольких клиниках в разные смены. В каждой клинике предусмотрен кабинет, в котором работают эксперты. Кроме того, законодательством регламентированы ограничения по режиму труда и отдыху рабочих - два выходных дня в неделю. 

*Резюме:* задача состоит в том, чтобы для каждого ведущего стоматолога назначить сменный график и место работы.


## Постановка задачи

Сеть из 3 стоматологических клиник имеет в своем ассортименте 5 экспертов. Все клиники сети работают без выходных по одной смене каждый день (8 часов). График работы сотрудников должен иметь недельный цикл (горизонт планирования 7 дней). Необходимо распределить сотрудников по клиникам и сменам при следующих ограничениях:

- Каждый эксперт может работать только в одной клинике в смену;
- В каждой клинике в любой из дней недели должен работать ровно один эксперт (одно помещение под эксперта); 
- Эксперты работают 5 смен в неделю (режим труда и отдыха);
- В каждой клинике в течение недели должны работать не менее 2 разных экспертов (имитация выбора для клиента).

![Визуализация задачи](https://drive.google.com/uc?export=view&id=1flZm_VR74JJXyGhDprOm_Jb5gV_lJBFZ)

<!-- - Эксперт 1 не может работать с экспертом 2 в одной клинике.  -->


## Построение модели и ее python реализация

Рассматриваемая нами задача состоит из набора ограничений, как и задача планирования расписаний в целом, ее можно представить в виде [задачи удовлетворения ограничений (ЗУО)](https://ru.wikipedia.org/wiki/%D0%A3%D0%B4%D0%BE%D0%B2%D0%BB%D0%B5%D1%82%D0%B2%D0%BE%D1%80%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BE%D0%B3%D1%80%D0%B0%D0%BD%D0%B8%D1%87%D0%B5%D0%BD%D0%B8%D0%B9). Элементы "языка ЗУО": 
- набор решающих переменных; 
- допустимые значения этих переменных;
- список ограничений, накладываемых на переменные;
- целевая функция. 

Представление задачи в формате ЗУО не дает ее решение, а только предлагает вариант формализации в некоторой математической форме. Одним из вариантов для ее решения является парадигма [программирование в ограничениях](https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%BE%D0%B3%D1%80%D0%B0%D0%BD%D0%B8%D1%87%D0%B5%D0%BD%D0%B8%D1%8F%D1%85) (Constraint Programming, CP). Она представляет собой набор методов и алгоритмов для решения ЗУО. 

Конечно, уже разработаны и упакованы библиотеки для решения задач удовлетворения ограничений, в которых реализованы алгоритмы программирования в ограничениях, такие пакеты называются solver или cp-solver. 

Из числа open source решений наиболее эффективным является cp-sat solver ORtools. Библиотека также предоставляет функционал для описания ЗУО. На примере данного solver`а будем моделировать и решать задачу планирования расписания смен ведущих стоматологов сети клиник. 

Установить библиотеку ORtools в среду python можно с помощью pip:

In [ ]:
%pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 31.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


### Индексы и входные данные

В задаче фигурируют три объекта: клиника, эксперт и смена. Введем для них следующие обозначения:

$k \in K$ - индекс и список клиник, клиника $k$ содержится во множестве $K$;

$e \in E$ - индекс и список экспертов, эксперт $e$ содержится во множестве $E$;

$t \in T$ - индекс и дни недели, день недели $t$ содержится во множестве $T$.

Запишем эти множества в виде списков Python:

In [ ]:
# Инициализируем множества/списки
K = ["Klinika1", "Klinika2", "Klinika3"]  # Список клиник в сети
E = ["Expert1", "Expert2", "Expert3", "Expert4", "Expert5"]  # Список экспертов
T = ["Smena" + str(t) for t in range(1, 8)]  # Список смен

trudovoy_kodex = 5  # Константа, ограничение кол-ва рабочих смен у эксперта
min_experts_per_clinic = 2  # Минимальное кол-во экспертов, привязанных к клинике

### Импорт библиотеки и инициализация модели

ORtools содержит в себе различные обертки для различных классов задач, в том числе для ЗУО. Поэтому импорт нужной нам части находится на четвертом уровне: `from ortools.sat.python import cp_model`. Переменные, ограничения и целевая функция инициализируются в экземпляре класса `CpModel()`. Объект модели может содержать только описание одной задачи, но никто не мешает инициализировать несколько экземпляров модели. 


In [ ]:
# Импорт "редактора" для записи ЗУО
from ortools.sat.python import cp_model
import pandas as pd

# Инициализация модели
model = cp_model.CpModel()

### Инициализация переменных

Прежде чем перейти к формулированию ограничений, определим набор решающих переменных. В задаче фигурируют три типа объектов/параметров: эксперт, клиника и смена. Все возможные комбинации этих объектов **клиника**-**эксперт**-**смена** представляют собой потенциальные варианты работы экспертов. 

Каждую такую комбинацию свяжем с переменной $x_{ket}$, которая принимает значение 1, если эксперт $e \in E$ работает в смену $t \in T$ в клинике $k \in K$, 0 - в противном случае. Таким образом, у нас есть 3\*5*7 = 105 переменных, соответствующих всем комбинациям. 

*Например:* если переменная для комбинации Klinika1-Expert1-Smena1 равна 1, следовательно, Эксперт 1 должен работать в Клинике 1 в Смену 1. Если значение переменной равно 0, то Эксперт 1 не работает в Клинике 1 в Смену 1.

Кроме допустимых комбинаций введем вспомогательные переменные-индикаторы **клиника**-**эксперт**. Значение переменной $y_{ke}$, равное 1, будет означать, что эксперт $e$ работает, по крайней мере, одну смену в клинике $k$, 0 - эксперт $e$ не работает в клинике $k$. 

*Например:* если переменная для комбинации Klinika1-Expert1 равна 1, следовательно, Эксперт 1 должен работать в Клинике 1 хотя бы одну смену. Если значение переменной равно 0, то Эксперт 1 не работает в Клинике 1 вообще.

Добавление переменных в `CpModel` возможно через метод `NewBoolVar`, если переменная бинарная (может принимать значения 0 или 1) или с помощью метода `NewIntVar` для целочисленных переменных. Переменные, соответствующие допустимым комбинациям, будем складировать в словарь `X`, переменные-индикаторы - в `Y`.

**Важно!** Использование cp-sat solver'а возможно только для решения задач с целочисленными переменными. 

In [ ]:
# Инициализация переменных комбинации
X = {}  # Словарь для хранения ссылок на переменные

for k in K:
  for e in E:
    for t in T:
      var_name = f"x_{k}_{e}_{t}"  # Название переменной
      X[k, e, t] = model.NewBoolVar(name=var_name)

print(f"Кол-во переменных комбинаций: {len(X)}")
print(f"Пример переменной в формате (ключ, переменная): {list(X.items())[0]}")

# Инициализация переменных-индикаторов
Y = {}  # Словарь для хранения ссылок на переменные-индикаторы

for k in K:
  for e in E:
    var_name = f"y_{k}_{e}"  # Название переменной-индикатора
    Y[k, e] = model.NewBoolVar(name=var_name)

Кол-во переменных комбинаций: 105
Пример переменной в формате (ключ, переменная): (('Klinika1', 'Expert1', 'Smena1'), x_Klinika1_Expert1_Smena1(0..1))


### Ограничение: один эксперт в каждой клинике каждый день

Всего в нашем распоряжении пять экспертов и три клиники. Ровно один из этих пяти экспертов должен работать в Клинике 1 в Смену 1. Это условие можно записать следующим образом:

$$x_{\text{Klinika1}, \text{Expert1}, \text{Smena1}} + x_{\text{Klinika1}, \text{Expert2}, \text{Smena1}} + x_{\text{Klinika1}, \text{Expert3}, \text{Smena1}} + $$ 
$$x_{\text{Klinika1}, \text{Expert4}, \text{Smena1}} + x_{\text{Klinika1}, \text{Expert5}, \text{Smena1}} = 1.$$

Аналогичные ограничения создаем для всех остальных пар **клиника**-**смена**. В связи с большим количеством ограничений (3\*7=21 шт.), текстовую запись всех ограничений опустим. 

Более лаконично ограничения можно записать в математической форме с использованием знака суммы $\sum$ и символа для каждого элемента множества $\forall$ («Для любого...»).

$$\sum_{e \in E} x_{ket} = 1, \quad \forall k \in K, \forall t \in T.$$

Добавление ограничений в `CpModel` возможно через метод `Add`, но наше ограничение подпадает под определенный шаблон ограничений `AddExactlyOne` (ровно одна переменная равняется 1). Использование таких шаблонов позволяет алгоритмам автоматически подстраиваться под ограничения и работать эффективнее. Добавим ограничения в модель. 

In [ ]:
# Ограничение: один эксперт в каждой клинике каждый день
for k in K:  # для каждой клиники
  for t in T:   # для каждого дня недели

    # Список экспертов для работы в клинике "k" в смену "t"
    lst_vars = [X[k, e, t] for e in E]  

    # Добавление ограничений в модель: ровно один эксперт в клинике в смену
    model.AddExactlyOne(lst_vars) 
    # model.Add(sum(lst_vars) == 1)  # Альтернативный способ добавления ограничения

### Ограничение: эксперт работает не более чем в одной клинике в смену

Исходя из физических ограничений, сотрудник не может находиться одновременно в двух разных местах. Данное ограничение очевидно, но в модель его необходимо передать совместно с другими ограничениями. 

В сети три клиники, в одной из которых может работать Эксперт 1 в Смену 1. Кроме того, Эксперт 1 может вообще не работать в Смену 1 - выходной. Это условие с помощью введенных переменных можно записать как 

$$x_{\text{Klinika1}, \text{Expert1}, \text{Smena1}} + x_{\text{Klinika2}, \text{Expert1}, \text{Smena1}} + x_{\text{Klinika3}, \text{Expert1}, \text{Smena1}}  \le 1.$$

Аналогичные ограничения создаем для всех остальных пар **эксперт**-**смена**. В связи с большим количеством ограничений (5*7=35 шт.) текстовую запись всех ограничений снова опустим. Лаконичная запись в виде формулы: 

$$\sum_{k \in K} x_{ket} \le 1, \quad \forall e \in E, \forall t \in T.$$

Данное ограничение является шаблонным, задается посредством метода `AddAtMostOne` (не более чем одна переменная равна 1).


In [ ]:
# Ограничение: Каждый эксперт может работать только в одной клинике в смену.
for e in E:  # для каждого эксперта
  for t in T:   # для каждой смены

    # Список клиник для работы эксперта "e" в смену "t"
    lst_vars = [X[k, e, t] for k in K]  

    # Добавление ограничений в модель: у эксперта не более одной клиники в смену
    model.AddAtMostOne(lst_vars) 
    # model.Add(sum(lst_vars) <= 1)  # Альтернативный способ добавления ограничения

### Ограничение: связь переменных-комбинаций и переменных-индикаторов

Данный тип ограничений является вспомогательным с целью упростить формализацию других ограничений. Само ограничение связывает переменные-индикаторы с переменными-комбинациями. Смысл ограничения в следующем: если Эксперт 1 работает в Клинике 1 хотя бы одну смену из семи, тогда Эксперт 1 связан (ассоциирован) с Клиникой 1, в противном случае не связан с Клиникой 1. Необходимость этого знания будет описана в следующем ограничении. 

С целью формализации ограничения воспользуемся функцией $\max()$, которая возвращает максимальное значение среди набора переменных. В нашем случае переменные могут принимать значения 0 или 1. Поэтому максимальное значение не может быть больше 1, что входит в диапазон допустимых значений переменной $y_{ke}$. Пример ограничений для Эксперта 1 и Клиники 1: 

$$\max(x_{\text{Klinika1}, \text{Expert1}, \text{Smena1}}; x_{\text{Klinika1}, \text{Expert1}, \text{Smena2}}; x_{\text{Klinika1}, \text{Expert1}, \text{Smena3}};$$ 

$$x_{\text{Klinika1}, \text{Expert1}, \text{Smena4}};x_{\text{Klinika1} \text{Expert1} \text{Smena5}}; x_{\text{Klinika1}, \text{Expert1}, \text{Smena6}};$$

$$x_{\text{Klinika1}, \text{Expert1}, \text{Smena7}}) = y_{\text{Klinika1}. \text{Expert1}}.$$


<!-- Для реализации данного условия потребуются два типа ограничений: верхняя граница и нижняя граница. Пример ограничений для Эксперта 1 и Клиники 1: 

Нижняя граница

$$x_{\text{Klinika1}, \text{Expert1}, \text{Smena1}} + x_{\text{Klinika1}, \text{Expert1}, \text{Smena2}} + x_{\text{Klinika1}, \text{Expert1}, \text{Smena3}} + $$ 

$$x_{\text{Klinika1}, \text{Expert1}, \text{Smena4}} + x_{\text{Klinika1}, \text{Expert1}, \text{Smena5}} + x_{\text{Klinika1}, \text{Expert1}, \text{Smena6}} +$$

$$x_{\text{Klinika1}, \text{Expert1}, \text{Smena7}} \ge y_{\text{Klinika1}, \text{Expert1}}.$$

Верхняя граница

$$x_{\text{Klinika1}, \text{Expert1}, \text{Smena1}} + x_{\text{Klinika1}, \text{Expert1}, \text{Smena2}} + x_{\text{Klinika1}, \text{Expert1}, \text{Smena3}} + $$ 

$$x_{\text{Klinika1}, \text{Expert1}, \text{Smena4}} + x_{\text{Klinika1}, \text{Expert1}, \text{Smena5}} + x_{\text{Klinika1}, \text{Expert1}, \text{Smena6}} +$$

$$x_{\text{Klinika1}, \text{Expert1}, \text{Smena7}} \le 5 * y_{\text{Klinika1}, \text{Expert1}},$$

где константа $5$ - это максимальное кол-во смен, которые Эксперт 1 может отработать за период.   -->

Если хотя бы одна из переменных $x_{ket}$ в $\max$ будет иметь значение 1, то результат функции $\max()$ будет равен 1. Тогда переменная-индикатор так же будет равна 1. А это означает, что можно продавать услуги эксперта $e$ в клинике $k$.

Эти ограничения гарантируют связь переменных-комбинаций и переменных-индикаторов. Лаконичная форма записи ограничений для **клиника**-**эксперт**:

$$\max_{t \in T}x_{ket} = y_{ke}, \quad \forall k \in K, \forall e \in E.$$

<!-- $$\sum_{t \in T} x_{ket} \ge y_{ke}, \quad \forall k \in K, \forall e \in E,$$

$$\sum_{t \in T} x_{ket} \le 5*y_{ke}, \quad \forall k \in K, \forall e \in E.$$ -->

В `CpModel()` метод `AddMaxEquality()` позволяет задать ограничения с функцией $\max$. В качестве аргументов необходимо передать `target` и `variables`, в нашем случае `target` - это переменная $y_{ke}$, а `variables` - список переменных $x_{ket}$, соответствующий всем возможным сменам эксперта $e$ в клинике $k$.

<!-- передать ограничения (self, target, variables) Данное ограничение является шаблонным, задается посредством метода AddAtMostOne (не более чем одна переменная равна 1). 
!!! Можно через модуль. -->

In [ ]:
# Ограничение (вспомогательное): индикатор работы эксперта "e" в клинике "k"
for k in K:
  for e in E:

    # Список смен для работы в клинике "k" и эксперта "e"
    lst_vars = [X[k, e, t] for t in T]  

    # Добавление ограничений в модель: индикатор работы эксперта "e" в клинике "k"
    model.AddMaxEquality(Y[k, e], lst_vars)
    # model.Add(sum(lst_vars) >= Y[k, e])  # Нижняя граница
    # model.Add(sum(lst_vars) <= expert_shift_lim * Y[k, e])  # Верхняя граница

### Ограничение: в каждой клинике не менее двух экспертов 

Бизнес-ограничение обусловлено спектром сервиса в виде предоставления клиентам возможности выбирать из нескольких экспертов. Запись ограничения для случая Клиника 1: 

$$y_{\text{Klinika1}, \text{Expert1}} + y_{\text{Klinika1}, \text{Expert2}} + y_{\text{Klinika1}, \text{Expert3}} + $$ 
$$y_{\text{Klinika1}, \text{Expert4}} + y_{\text{Klinika1}, \text{Expert5}} \ge 2.$$

Заметим, что в ограничении участвуют вспомогательные переменные-индикаторы, которые отражают привязку эксперта к клинике, а само ограничение гарантирует привязку минимум двух разных экспертов. 

Лаконичная форма записи ограничений для каждой **клиники**:

$$\sum_{e \in E} y_{ke} \ge 2, \quad \forall k \in K.$$

In [ ]:
# Ограничение: в каждой клинике не менее двух экспертов 
for k in K:

  # Список экспертов, которые могут работать в клинике "k"
  lst_vars = [Y[k, e] for e in E]  

  # Добавление ограничений в модель: минимум 2 разных эксперта должны работать в клинике "k"
  model.Add(sum(lst_vars) >= min_experts_per_clinic)

### Ограничение: эксперт может работать пять смен в неделю

Источник ограничения - Трудовой кодекс. Модель должна понимать, что установлено ограничение на кол-во рабочих дней в неделю, обусловленное законом. Каждый эксперт суммарно во всех клиниках не может работать более 5 смен в неделю. Пример ограничения громоздкий, так как содержит 3*7=21 переменную, поэтому запишем его в математической форме:

$$0 \le \sum_{k \in K} \sum_{t \in T} x_{ket} \le 5, \quad \forall e \in E.$$

Ограничение является линейным, поэтому добавим ограничение с помощью метода `AddLinearConstraint()`. В качестве аргументов необходимо передать линейное выражение, в нашем случае - это левая часть ограничения, нижнюю и верхнюю границы выражения, т. е. 0 и 5 для нашего примера.

In [ ]:
# Ограничение: эксперт может работать не более 5 смен
for e in E:

  # Список рабочих смен эксперта "e" во всех клиниках
  lst_vars = [X[k, e, t] for k in K for t in T]

  # Добавление ограничений в модель: 0 <= кол-во смен у эксперта <= 5
  model.AddLinearConstraint(sum(lst_vars), 0, trudovoy_kodex)

### Доп.ограничение: конфликт Эксперта 1 и Эксперта 2

Данное ограничение призвано продемонстрировать возможность учета межличностных отношений на примере конфликта между Экспертом 1 и Экспертом 2. Ограничение подразумевает, что если Эксперт 1 и Эксперт 2 будут пересекаться по клинике, то они намеренно будут портить оборудование или рабочее место для организации дискомфорта рабочего процесса друг друга. Ограничение для Клиники 1 запишется как 

$$y_{\text{Klinika1}, \text{Expert1}} \land y_{\text{Klinika1}, \text{Expert2}} = 0.$$

Объект модели позволяет записать логические ограничения "И" с помощью метода 

In [ ]:
# Ограничение: конфликт между Экспертом 1 и Экспертом 2
for k in K:

  # Добавление ограничений в модель: Экспертом 1 не может работать в клинике с Экспертом 2
  # Ограничение сформулировано через отрицание исходного. 
  model.AddBoolOr([Y[k, "Expert1"].Not(), Y[k, "Expert2"].Not()])

### Поиск решения

Инициализируем solver и запускаем поиск решения поставленной задачи:

In [ ]:
# Инициализация solver
solver = cp_model.CpSolver()

# Решение задачи - та самая одна строчка
status = solver.Solve(model)

# Проверяем статус
if status == cp_model.FEASIBLE:
  print('Найдено решение')

Извлечем полученное решение из модели, для этого запрашиваем у solver значение переменной с помощью метода `Value()`:

In [ ]:
# Извлекаем результат (решение)
result = {}
for key, val in X.items():
  sol = solver.Value(val)  # Достаем значение переменной
  if sol > 0:  # Собираем только те переменные, которые имеют значение 1
    result[key] = 1

# Извлекаем значения переменных индикаторов
resulty = {}
for key, val in Y.items():
  sol = solver.Value(val)
  if sol > 0:
    resulty[key] = 1

print(resulty)
result

{('Klinika1', 'Expert3'): 1, ('Klinika1', 'Expert4'): 1, ('Klinika2', 'Expert1'): 1, ('Klinika2', 'Expert2'): 1, ('Klinika2', 'Expert3'): 1, ('Klinika2', 'Expert4'): 1, ('Klinika2', 'Expert5'): 1, ('Klinika3', 'Expert1'): 1, ('Klinika3', 'Expert2'): 1, ('Klinika3', 'Expert5'): 1}


{('Klinika1', 'Expert3', 'Smena2'): 1,
 ('Klinika1', 'Expert3', 'Smena5'): 1,
 ('Klinika1', 'Expert3', 'Smena6'): 1,
 ('Klinika1', 'Expert4', 'Smena1'): 1,
 ('Klinika1', 'Expert4', 'Smena3'): 1,
 ('Klinika1', 'Expert4', 'Smena4'): 1,
 ('Klinika1', 'Expert4', 'Smena7'): 1,
 ('Klinika2', 'Expert1', 'Smena4'): 1,
 ('Klinika2', 'Expert2', 'Smena2'): 1,
 ('Klinika2', 'Expert2', 'Smena5'): 1,
 ('Klinika2', 'Expert3', 'Smena3'): 1,
 ('Klinika2', 'Expert3', 'Smena7'): 1,
 ('Klinika2', 'Expert4', 'Smena6'): 1,
 ('Klinika2', 'Expert5', 'Smena1'): 1,
 ('Klinika3', 'Expert1', 'Smena5'): 1,
 ('Klinika3', 'Expert2', 'Smena1'): 1,
 ('Klinika3', 'Expert2', 'Smena4'): 1,
 ('Klinika3', 'Expert5', 'Smena2'): 1,
 ('Klinika3', 'Expert5', 'Smena3'): 1,
 ('Klinika3', 'Expert5', 'Smena6'): 1,
 ('Klinika3', 'Expert5', 'Smena7'): 1}

<!-- Диаграмма Гантта позволяет локанично отобразить полученное решение. Из диаграммы видно, что соблюдены все условия задачи.  -->
Полученное расписание удобно анализировать и использовать в табличном виде или в виде диаграммы Ганта. Маленький размер задачи и наглядная визуализация решения позволяют убедиться в выполнении всех заданных ограничений. Само решение при повторном запуске solver может отличаться, т.к. допустимых решений, удовлетворяющих заданным ограничениям, может быть много. 

![График работы сотрудников](https://drive.google.com/uc?export=view&id=1g4lE0-1tmDPdVfAp3oWnucaEcZf9-XM6)

Анализируя результат, можно заметить, что не все эксперты работают 5 смен. Это связано с тем, что объем требуемых смен для экспертов равен 3\*7=21, а максимальное кол-во смен пяти экспертов 5\*5 = 25. Таким образом, получаем, что 4 смены экспертов не используются (лишние).  

## Визуализация результата

Библиотека plotly позволяет быстро и комфортно отрисовать диаграмму Ганта полученного расписания. 

In [ ]:
%pip install plotly  # Установка библиотеки 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Конвертируем данные в таблицу pandas.DataFrame.

In [ ]:
import pandas as pd

# Переводим словарь результата в табличный вид
df_tmp = pd.DataFrame(list(result.keys()), columns=["Клиники", "Эксперты", "shift"])
df_tmp = df_tmp.sort_values("Эксперты")

# Извлекаем номер смены в виде целого числа
df_tmp["shift_int"] = df_tmp["shift"].apply(lambda x: int(x[-1]))

Строим диаграмму. Определим начало периода работы по построенному графику как 01.05.2023 и начало рабочей смены в 9:00. 

In [ ]:
import plotly.express as px
from datetime import datetime

# Инициализируем дату начала периода
bop = datetime(year=2023, month=5, day=1, hour=2)

# Преобразуем смены в даты
df_tmp["shift_s"] = bop + pd.to_timedelta(df_tmp["shift_int"], unit='D')
df_tmp["shift_e"] = bop + pd.to_timedelta(df_tmp["shift_int"], unit='D') + pd.to_timedelta(20, unit='H')

# строим диаграмму Гантта
fig = px.timeline(df_tmp, x_start="shift_s", x_end="shift_e", y="Клиники", color="Эксперты")
fig.update_yaxes(autorange="reversed")

# Вывод полученной диаграммы
fig.show()  

# Сохранение результата в html
fig.write_html("result.html")  

## Расширение задачи

Описанная выше математическая модель позволяет продемонстрировать основные методы и типы ограничений для моделирования. Но в реальности ограничений больше. Предлагаю попробовать самостоятельно доработать модель:

1. Добавить равномерность распределения нагрузки на экспертов;
2. Убрать из модели одного эксперта и добавить целевую функцию максимизации покрытия рабочих смен (кол-во смен 3\*7=21, а рабочие смены четырех экспертов 4\*5=20. Одну смену не удастся покрыть);
3. Добавить ограничение непрерывности выходных дней, чтобы эксперт отдыхал два дня подряд без перерыва; 
4. Добавить целевую функцию минимизации расстояния от места жительства эксперта до клиники. 

## Заключение

Резюмирую содержание обучающего материала: построили математическую модель планирования сменного графика для ведущих специалистов сети стоматологических клиник. Разработали программный прототип на базе Python пакета OR-Tools и решили задачу с помощью cp-sat solver.

## Ссылки
- Ссылка на расширенный материал в Jupyter Notebook;
- [Документация](https://developers.google.com/optimization/reference/python/index_python) Python библиотеки OR-Tools;
- Задача планирования расписаний из других [уст](https://habr.com/ru/companies/billing/articles/342550/); 
- Задача планирования расписаний из [примеров](https://developers.google.com/optimization/scheduling/employee_scheduling?hl=ru) OR-Tool. 

[<= Предыдущая статья](https://habr.com/ru/articles/731006/)

P.S. Направляйте оптимизационные задачи, которые хотелось бы увидеть в разборе.

P.S.S. Есть на примете статьи с моделями в PuLP или ORtools? Присылайте, дополню ссылками.

*Aleksejs Lozkins*